---

_You are currently looking at **version 1.0** of this notebook. To download notebooks and datafiles, as well as get help on Jupyter notebooks in the Coursera platform, visit the [Jupyter Notebook FAQ](https://www.coursera.org/learn/python-text-mining/resources/d9pwm) course resource._

---

# Assignment 1

In this assignment, you'll be working with messy medical data and using regex to extract relevant infromation from the data. 

Each line of the `dates.txt` file corresponds to a medical note. Each note has a date that needs to be extracted, but each date is encoded in one of many formats.

The goal of this assignment is to correctly identify all of the different date variants encoded in this dataset and to properly normalize and sort the dates. 

Here is a list of some of the variants you might encounter in this dataset:
* 04/20/2009; 04/20/09; 4/20/09; 4/3/09
* Mar-20-2009; Mar 20, 2009; March 20, 2009;  Mar. 20, 2009; Mar 20 2009;
* 20 Mar 2009; 20 March 2009; 20 Mar. 2009; 20 March, 2009
* Mar 20th, 2009; Mar 21st, 2009; Mar 22nd, 2009
* Feb 2009; Sep 2009; Oct 2010
* 6/2008; 12/2009
* 2009; 2010

Once you have extracted these date patterns from the text, the next step is to sort them in ascending chronological order accoring to the following rules:
* Assume all dates in xx/xx/xx format are mm/dd/yy
* Assume all dates where year is encoded in only two digits are years from the 1900's (e.g. 1/5/89 is January 5th, 1989)
* If the day is missing (e.g. 9/2009), assume it is the first day of the month (e.g. September 1, 2009).
* If the month is missing (e.g. 2010), assume it is the first of January of that year (e.g. January 1, 2010).

With these rules in mind, find the correct date in each note and return a pandas Series in chronological order of the original Series' indices.

For example if the original series was this:

    0    1999
    1    2010
    2    1978
    3    2015
    4    1985

Your function should return this:

    0    2
    1    4
    2    0
    3    1
    4    3

Your score will be calculated using [Kendall's tau](https://en.wikipedia.org/wiki/Kendall_rank_correlation_coefficient), a correlation measure for ordinal data.

*This function should return a Series of length 500 and dtype int.*

In [1]:
import pandas as pd
import numpy as np

doc = []
with open('dates.txt') as file:
    for line in file:
        doc.append(line)

df = pd.Series(doc)
df.head(10)

0         03/25/93 Total time of visit (in minutes):\n
1                       6/18/85 Primary Care Doctor:\n
2    sshe plans to move as of 7/8/71 In-Home Servic...
3                7 on 9/27/75 Audit C Score Current:\n
4    2/6/96 sleep studyPain Treatment Pain Level (N...
5                    .Per 7/06/79 Movement D/O note:\n
6    4, 5/18/78 Patient's thoughts about current su...
7    10/24/89 CPT Code: 90801 - Psychiatric Diagnos...
8                         3/7/86 SOS-10 Total Score:\n
9             (4/10/71)Score-1Audit C Score Current:\n
dtype: object

In [5]:
def date_sorter():

    import re    
    # my patterns
    regexp = {
        1:re.compile(r'(?P<month>[0-9]{1,2})[/-](?P<day>[0-9]{1,2})[/-](?P<year>[0-9]{2,4})'),
        2:re.compile(r'(?P<day>(?:[0-9]{1,2})) (?P<month>(?:Jan|Feb|Mar|Apr|May|Jun|Jul|Aug|Sep|Oct|Nov|Dec)) (?P<year>[0-9]{4})'),
        3:re.compile(r'(?P<month>(?:Jan|Feb|Mar|Apr|May|Jun|Jul|Aug|Sep|Oct|Nov|Dec)).? (?P<day>(?:[0-9]{2}))[ ,.]*(?P<year>[0-9]{4})'),
        4:re.compile(r'(?P<month>(?:Jan|Feb|Mar|Apr|May|Jun|Jul|Aug|Sep|Oct|Nov|Dec)) (?P<year>[0-9]{4})'),
        5:re.compile(r'(?P<month>[0-9]{1,2})[/-](?P<year>[0-9]{4})'),
        6:re.compile(r'(?P<year>[12][0-9]{3})')
        }

    months = dict(Jan=1, Feb=2, Mar=3, Apr=4, May=5, Jun=6, Jul=7, Aug=8, Sep=9, Oct=10, Nov=11,Dec=12)
    
    # make it a DataFrame
    myFrame = df.to_frame().reset_index()
    
    # weird without column name
    myFrame.rename(columns ={0: 'text'}, inplace =True)
    
    # pre-processing
    # replace st, nd, rd, th with spaces
    myFrame['text'] = myFrame['text'].str.replace(r'\d{1,2}(st|nd|rd|th),',"")
    # remove other unwanted things
    myFrame['text'] = myFrame['text'].str.replace(r'[:\n)(.,]',"")    
    # bring all long names to short names -> one time parse
    myFrame['text'] = myFrame['text'].str.replace(r'(Jan|Feb|Mar|Apr|May|Jun|Jul|Aug|Sep|Oct|Nov|Dec)[a-z]*', lambda x: x.groups()[0][:3])
    
    # initializing
    myList=list()

    for i in range(0, len(myFrame)):
    #for i in {405}: #345 
        for key,pattern in regexp.items():
            try:
                m = pattern.search(myFrame.iloc[i]['text'])
                #print(i,m,myFrame.iloc[i]['text'], pattern)
                if m:
                    myList.append([m.groupdict() for m in pattern.finditer(myFrame.iloc[i]['text'])])                 
                    #print(myList)
                    break
            except ValueError:
                pass

    myResult = pd.concat([pd.DataFrame(l) for l in myList],axis=0).reset_index()
    
    myResult['month']=np.where(myResult['month'].str.match(r'[a-zA-z]{3}'),myResult['month'].map(months),myResult['month'])
    
    myResult = myResult.fillna(1)
    # make numbers, could be loop over all columns
    myResult['month'] = pd.to_numeric(myResult['month'])
    myResult['day'] = pd.to_numeric(myResult['day'])
    myResult['year'] = pd.to_numeric(myResult['year'])
    
    # add 1900 for all years <100
    myResult['year'] = np.where(myResult['year']<100, myResult['year']+1900, myResult['year'])
    
    # filter fakes
    myResult = myResult[myResult.year > 1900]
    # get index straight again 
    myResult.reset_index(inplace=True, drop=True)
    
    myResult['ind']= myResult.index
    #order by dates, keep index column
    myResult = myResult.sort_values(['year','month','day','ind'], ascending=True).reset_index()
    
    return myResult['ind']

0        9
1       84
2        2
3       53
4       28
5      474
6      153
7       13
8      129
9       98
10     111
11     225
12      31
13     171
14     191
15     486
16     335
17     415
18      36
19     323
20     405
21     422
22     375
23     380
24     345
25      57
26     481
27     436
28     104
29     299
      ... 
470    220
471    208
472    243
473    139
474    320
475    383
476    244
477    286
478    480
479    431
480    279
481    198
482    381
483    463
484    366
485    255
486    439
487    401
488    475
489    257
490    152
491    235
492    464
493    253
494    231
495    427
496    141
497    186
498    161
499    413
Name: ind, Length: 500, dtype: int64